# Export aggregated and non-aggregated splits on data folder

In [1]:
import os
import datasets

/Users/prl222/opt/anaconda3/envs/data-challenge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download dataset from the Huggingface Hub

In [2]:
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')
df = dataset['train'].to_pandas()

Found cached dataset parquet (/Users/prl222/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.11it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment_id', 'annotator_id', 'platform', 'sentiment', 'respect', 'insult', 'humiliate', 'status', 'dehumanize', 'violence', 'genocide', 'attack_defend', 'hatespeech', 'hate_speech_score', 'text', 'infitms', 'outfitms', 'annotator_severity', 'std_err', 'annotator_infitms', 'annotator_outfitms', 'hypothesis', 'target_race_asian', 'target_race_black', 'target_race_latinx', 'target_race_middle_eastern', 'target_race_native_american', 'target_race_pacific_islander', 'target_race_white', 'target_race_other', 'target_race', 'target_religion_atheist', 'target_religion_buddhist', 'target_religion_christian', 'target_religion_hindu', 'target_religion_jewish', 'target_religion_mormon', 'target_religion_muslim', 'target_religion_other', 'target_religion', 'target_origin_immigrant', 'target_origin_migrant_worker', 'target_origin_specific_country', 'target_origin_undocumented', 'target_origin_other', 'target_origin', 'target_gender_men', 'target

## Prepare data

In [4]:
# little insights about the data
print(df.shape)
print(df.columns.to_list()[:22])
print(df.columns.to_list()[22:])
text_col, id_col = 'text', 'comment_id'
df.head(2)

(135556, 131)
['comment_id', 'annotator_id', 'platform', 'sentiment', 'respect', 'insult', 'humiliate', 'status', 'dehumanize', 'violence', 'genocide', 'attack_defend', 'hatespeech', 'hate_speech_score', 'text', 'infitms', 'outfitms', 'annotator_severity', 'std_err', 'annotator_infitms', 'annotator_outfitms', 'hypothesis']
['target_race_asian', 'target_race_black', 'target_race_latinx', 'target_race_middle_eastern', 'target_race_native_american', 'target_race_pacific_islander', 'target_race_white', 'target_race_other', 'target_race', 'target_religion_atheist', 'target_religion_buddhist', 'target_religion_christian', 'target_religion_hindu', 'target_religion_jewish', 'target_religion_mormon', 'target_religion_muslim', 'target_religion_other', 'target_religion', 'target_origin_immigrant', 'target_origin_migrant_worker', 'target_origin_specific_country', 'target_origin_undocumented', 'target_origin_other', 'target_origin', 'target_gender_men', 'target_gender_non_binary', 'target_gender_tr

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other
0,47777,10873,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,39773,2790,2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False


In [5]:
# aggregate data 
comments = df[[id_col, text_col]].drop_duplicates().sort_values(id_col)
print(comments.shape)
model_outputs = ['sentiment', 'respect', 'insult', 'humiliate', 'status', 'dehumanize', 'violence', 'genocide', 'attack_defend', 'hatespeech', 'hate_speech_score']
target_identities = ['target_race_asian', 'target_race_black', 'target_race_latinx', 'target_race_middle_eastern', 'target_race_native_american', 'target_race_pacific_islander', 'target_race_white', 'target_race_other', 'target_race', 
                     'target_religion_atheist', 'target_religion_buddhist', 'target_religion_christian', 'target_religion_hindu', 'target_religion_jewish', 'target_religion_mormon', 'target_religion_muslim', 'target_religion_other', 'target_religion', 
                     'target_origin_immigrant', 'target_origin_migrant_worker', 'target_origin_specific_country', 'target_origin_undocumented', 'target_origin_other', 'target_origin', 
                     'target_gender_men', 'target_gender_non_binary', 'target_gender_transgender_men', 'target_gender_transgender_unspecified', 'target_gender_transgender_women', 'target_gender_women', 'target_gender_other', 'target_gender', 
                     'target_sexuality_bisexual', 'target_sexuality_gay', 'target_sexuality_lesbian', 'target_sexuality_straight', 'target_sexuality_other', 'target_sexuality', 
                     'target_age_children', 'target_age_teenagers', 'target_age_young_adults', 'target_age_middle_aged', 'target_age_seniors', 'target_age_other', 'target_age', 
                     'target_disability_physical', 'target_disability_cognitive', 'target_disability_neurological', 'target_disability_visually_impaired', 'target_disability_hearing_impaired', 'target_disability_unspecific', 'target_disability_other', 'target_disability']
agreement = df[[id_col] + model_outputs + target_identities].groupby(id_col).agg('mean')
print(agreement.shape)
#targets = agreement[target_identities]
#threshold = 0.5
#df_aggregated = (targets >= threshold).astype('int').reset_index(level=0).merge(right=comments, how='left')
df_aggregated = agreement.reset_index(level=0).merge(right=comments, how='left')
#print(df_aggregated.shape)
#df_aggregated[model_outputs] = agreement[model_outputs].values
print(df_aggregated.shape)
df_aggregated.head(2)

(39565, 2)
(39565, 64)
(39565, 66)


,comment_id,sentiment,respect,insult,humiliate,status,dehumanize,violence,genocide,attack_defend,...,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability,text
0,1,3.750000,3.500000,3.250000,2.750000,3.250000,2.750000,0.250000,0.0,3.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,White people are trash.
1,2,3.333333,3.333333,3.333333,2.666667,2.666667,1.666667,0.666667,0.0,2.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"You've caught the big gay, it's known to be ai..."


set(df_aggregated.target_gender.values)

In paper they use values as y_soft and the weights (sqrt of the values)

## Create splits

set(df_aggregated.hatespeech.values)

In [6]:
# Select 30% as heldout set

from sklearn.model_selection import train_test_split

x, y = df_aggregated[id_col].values, df_aggregated['hate_speech_score'].values
x_train, x_test, _, _ = train_test_split(x, y, shuffle=True, random_state=4, test_size=0.33)

train, test = df.loc[df[id_col].isin(x_train)].copy(), df.loc[df[id_col].isin(x_test)].copy()
train_aggregated, test_aggregated = df_aggregated.loc[df_aggregated[id_col].isin(x_train)].copy(), df_aggregated.loc[df_aggregated[id_col].isin(x_test)].copy()

print('Non-aggregated data: {}. {} (train) + {} (test) = {}'.format(df.shape[0], train.shape[0], test.shape[0], train.shape[0]+test.shape[0]))
print('Aggregated: {} = {} (train) + {} (test)'.format(df_aggregated.shape[0], train_aggregated.shape[0], test_aggregated.shape[0]))


Non-aggregated data: 135556. 93819 (train) + 41737 (test) = 135556
Aggregated: 39565 = 26508 (train) + 13057 (test)


In [7]:
# unique IDs in the aggregated dataset
len(set(df_aggregated[id_col]))

39565

In [8]:
len(set(train_aggregated[id_col]))

26508

In [9]:
len(set(test_aggregated[id_col]))

13057

## Export training data (for participants)

In [10]:
# Export csv with all in 70% (train.csv) and 30% without labels and aggregated by sqrt/look in paper (test.csv) plus the hatecheck
data = {'train': train, 'train_aggregate': train_aggregated}
for dname, d in data.items():
    d.to_csv('../challenge/data/'+dname+'.csv', index=False)

## Export testing data for stress testing the models

In [11]:
# download repo
!git clone https://github.com/Ago3/Adversifier.git

Cloning into 'Adversifier'...
remote: Enumerating objects: 840, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 840 (delta 7), reused 3 (delta 3), pack-reused 821
Receiving objects: 100% (840/840), 1.59 MiB | 3.51 MiB/s, done.
Resolving deltas: 100% (508/508), done.


In [12]:
# creating docker image (ensure Docker is running)
os.chdir('Adversifier')
!pwd
!docker build -f AAA-Dockerfile -t aaa .

/Users/prl222/OneDrive-University/nobias-data-challenge/preparation/Adversifier
[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/2)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for dock

[+] Building 2.4s (2/3)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8              2.2s
[+] Building 2.6s (2/3)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8              2.4s
[+] Building 2.7s (2/3)               

[+] Building 4.7s (2/3)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8              4.5s
[+] Building 4.8s (2/3)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8              4.7s
[+] Building 5.0s (2/3)               

[+] Building 6.9s (2/3)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8              6.8s
[+] Building 7.1s (2/3)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8              6.9s
[+] Building 7.2s (2/3)               

[+] Building 9.2s (2/3)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8              9.0s
[+] Building 9.3s (2/3)                                                         
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8              9.2s
[+] Building 9.5s (2/3)               

[+] Building 11.4s (2/3)                                                        
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8             11.3s
[+] Building 11.6s (2/3)                                                        
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8             11.4s
[+] Building 11.7s (2/3)              

[+] Building 13.7s (2/3)                                                        
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8             13.5s
[+] Building 13.9s (2/3)                                                        
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8             13.7s
[+] Building 14.0s (2/3)              

[+] Building 16.0s (2/3)                                                        
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8             15.8s
[+] Building 16.1s (2/3)                                                        
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8             16.0s
[+] Building 16.3s (2/3)              

[+] Building 18.2s (3/4)                                                        
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docker.io/library/python:3.8             18.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
[+] Building 18.3s (9/10)                                                       
 => [internal] load build definition from AAA-Dockerfile                   0.1s
 => => transferring dockerfile: 170B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 157B                                          0.0s
 => [internal] load metadata for docke

In [13]:
# exporting train and test files for creating AAA adversarials (where only text colum  will be uploaded to 'test files' folder)
aaa_data = {'train': train_aggregated.copy(), 'test': test_aggregated.copy()}

os.chdir('../')
os.mkdir('Adversifier/mhs')
for set, d in aaa_data.items():
    d['hate_speech_score_bin'] = (d['hate_speech_score'] > 0.5).astype('int')
    print(d[['hate_speech_score','hate_speech_score_bin']].head(10))
    d.rename(columns={text_col: 'post_text', 'hate_speech_score_bin':'label'}, inplace=True)
    d[['post_text', 'label']].to_csv(f'Adversifier/mhs/aaa_{set}.csv', sep ='\t', index=False, header=False)

    hate_speech_score  hate_speech_score_bin
1                0.03                      0
2               -1.29                      0
3               -0.24                      0
6               -0.77                      0
8                1.35                      1
12              -3.28                      0
13               1.17                      1
14              -1.65                      0
15              -2.76                      0
17               2.83                      1
    hate_speech_score  hate_speech_score_bin
0                0.46                      0
4               -2.84                      0
5                1.72                      1
7                1.24                      1
9                1.07                      1
10               0.76                      1
11              -3.42                      0
16               2.50                      1
19              -4.85                      0
22              -1.52                      0


In [14]:
# Test 1 files should/will be the adversaries (not using F1 scores, this scores to rank and HateCheck to get more insights)
#test_aggregated[[id_col, text_col]].to_csv('test_1.csv',index=False)